# DOF and quadrature point plotter

This notebook helps visualize the locations of MFEM's DOFs and quadrature points for various element types and quadrature rules.

To use this, your python environment must have:
* PyMFEM (https://github.com/mfem/PyMFEM)
* Matplotlib (https://matplotlib.org)

## Preamble: import any necessary libs

In [ ]:
import mfem.ser as mfem

import numpy as np
import os
import math

%matplotlib inline
import matplotlib.pyplot as plt

%config InlineBackend.figure_formats = ['svg']


# Create some output directories for images
for d in ["figs", "figs/dofs", "figs/qpts", 
          "figs/dofs/svg", "figs/dofs/pdf", "figs/dofs/png",
          "figs/qpts/svg", "figs/qpts/pdf", "figs/qpts/png"]:
    if not os.path.exists(d):
        os.makedirs(d)

## Setup mesh and options

Create an mfem mesh and define some parameters

We currently support a simple Cartesian 2D mesh.
This can be easily extended to load a mesh from file.

In [ ]:
# Options for Cartesian mesh constructor are: 
# 0D: POINT
# 1D: SEGMENT 
# 2D: TRIANGLE, QUADRILATERAL
# 3D: TETRAHEDRON, HEXAHEDRON, WEDGE

mesh = mfem.Mesh(1,1,"QUADRILATERAL")
mesh.EnsureNodes()    

In [ ]:
# Define some parameters for the generated figures

dim = mesh.Dimension()
max_order = 6

# Polynomial order of the FECollection
orders = [i for i in range(max_order)]

# Basis types of the FiniteElementCollections
b_types = [ mfem.BasisType.GaussLobatto,
           mfem.BasisType.GaussLegendre,
           mfem.BasisType.Positive ]

# Quadrature types for the integration rules
q_types = [{'q': mfem.Quadrature1D.GaussLegendre, 'name': 'Gauss-Legendre'},
           {'q': mfem.Quadrature1D.GaussLobatto, 'name': 'Gauss-Lobatto'},
           {'q': mfem.Quadrature1D.OpenUniform, 'name': 'Open-Uniform'},
           {'q': mfem.Quadrature1D.OpenHalfUniform, 'name': 'Open-Half-Uniform'},
           {'q': mfem.Quadrature1D.ClosedGL, 'name': 'Closed-Gauss-Legendre'},
           {'q': mfem.Quadrature1D.ClosedUniform, 'name': 'Closed-Uniform'}]

# Finite Element collection types
fec_types = [ "H1", "L2" ]

## Functions to extract integration points and map them to physical space

In [ ]:
def getQptPositions(mesh, eid, ir):
    """Returns list of dictionaries of points in physical space for an element and integration rule.
    
       The dictionaries have entries for physical space position ('x', 'y')
       and reference space position and weight ('ix', 'iy', 'w').
    """
    
    t = mesh.GetElementTransformation(eid)

    pts = []
    mfem.Vector(3)
    for i in range(ir.GetNPoints()):
        ip = ir.IntPoint(i)
        v = t.Transform(ip)

        d = {'x' : v[0], 
             'y' : v[1], 
             'ix': ip.x, 
             'iy': ip.y,
             'w' : ip.weight}
        #print(d)
        pts.append(d)

    return pts

In [ ]:
def getDofPositions(fespace, eid):
    """Returns a list of dictionaries containing the DOF positions 
    in physical and reference space"""
    
    mesh = fespace.GetMesh()
    fe = fespace.GetFE(eid)
    ir = fe.GetNodes()
    
    return getQptPositions(mesh, eid, ir)

## Functions to generate plots for DOFs and quadrature points

In [ ]:
def plotDofPositions(name, pts, wscale = .1):
    """Creates a matplotlib plot for the DOFs in pts
    
       Note: Currently assumes all points are in a unit square.
       TODO: Extend this to draw curved elements of the mesh.
    """
    

    plt.clf()

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim((-.1, 1.1))
    ax.set_ylim((-.1, 1.1))

    rect = plt.Rectangle((0, 0), 1, 1, linewidth=3, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

    for p in pts:
        circle=plt.Circle((p['x'],p['y']),  wscale, facecolor='#c0504d', edgecolor='#366092')
        ax.add_patch(circle)

    plt.axis('off')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    print(F"Saving DOFs for '{name}'")
    fig.savefig(F'figs/dofs/svg/{name}.svg') # bbox_inches=extent ?
    fig.savefig(F'figs/dofs/png/{name}.png')
    fig.savefig(F'figs/dofs/pdf/{name}.pdf')

    plt.show()


In [ ]:
def plotQptPositions(name, pts, wscale = .1, use_weights = True, min_size = None):
    """Creates a matplotlib plot for the quadrature points in pts
    
       Note: Currently assumes all points are in a unit square.
       TODO: Extend this to draw curved elements of the mesh.
       
       Parameters:
         name          The name for the output figures
         pts           A collection of dictionaries of point data
         wscale        Used to scale the quadrature weights
         use_weights   When true, use the quadrature weights to scale the quadrature points
         min_size      Optionally sets a lower bound on the size of the quadrature points
    """

    plt.clf()

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim((-.1, 1.1))
    ax.set_ylim((-.1, 1.1))

    rect = plt.Rectangle((0, 0), 1, 1, linewidth=1.5, edgecolor='#969696', facecolor='none')
    ax.add_patch(rect)

    for p in pts:
        # Color depends on sign
        facecolor = '#0871b7C0' if (p['w'] >= 0) else '#B74E08C0'
        # Scale w/ weights proportional to area
        sc = wscale * math.sqrt(abs(p['w'])) if use_weights else wscale
        # Apply threshold to size, if applicable 
        sc = min_size if (min_size and sc < min_size) else sc
        
        circle=plt.Circle((p['x'],p['y']),  sc, facecolor=facecolor, edgecolor='#231f20')
        ax.add_patch(circle)

    plt.axis('off')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    print(F"Saving qpts for '{name}'")
    fig.savefig(F'figs/qpts/svg/{name}.svg') # bbox_inches=extent ?
    fig.savefig(F'figs/qpts/png/{name}.png')
    fig.savefig(F'figs/qpts/pdf/{name}.pdf')

    plt.show()

## Plot the DOFs

In [ ]:
# Create a grid function for each FE type, order and basis type and plot figures
# Skip the invalid combinations

for f in fec_types:
    for b in b_types:
        for p in orders:
            try:
                if "H1" in f:
                    fec = mfem.H1_FECollection(p, dim, b)
                elif "L2" in f:
                    fec = mfem.L2_FECollection(p, dim, b)
                fespace = mfem.FiniteElementSpace(mesh, fec)      
                
                bname = mfem.BasisType.Name(b).split(" ")[0]
                print(F"Working on {fec.Name()} -- dim {dim} -- basis type {bname} -- fec type {f}" )

                pts = getDofPositions(fespace, 0)
                plotDofPositions(F'{fec.Name()}_{bname}', pts, 0.05)
            except:
                #print(F"\tFEC {fec.Name()} did not work: dim {dim} -- basis type {b} -- fec type {f}" )
                pass
            

## Plot the quadrature points

In [ ]:
# Create a chart for each quadrature type and order defined above
# Skip the invalid combinations
# Note: Rules for 2*order and 2*order+1 are the same, so only plot the even ones

# Currently hard-coded for squares
g_type = mfem.Geometry.SQUARE
g_name = "square"

wscale=.125

for q in q_types:
    for o in orders:
        try:
            intrules = mfem.IntegrationRules(0, q['q'])
            ir = intrules.Get(g_type, 2*o)
            pts = getQptPositions(mesh, 0, ir)
            pts = sorted(pts, key = lambda p: (p['x']-.5)**2 + (p['y']-.5)**2, reverse=True)
                        
            name = F"qpts_{g_name}_{q['name']}_{dim}D_P{2*o}"

            #for p in pts:
            #    print(F"{name}: P{2*o} {p['w']}")
            
            plotQptPositions(name, pts, wscale, True, min_size=None)
            
        except:
            pass
